## Imports, Connect to Database

In [24]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [25]:
credFiles= '/data/Github/VaultBrain/credFiles_aws.yaml'
#'/Users/kuiqian/Github/VaultBrain/credFiles.yaml'
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']
dj.conn()

./lib/utilities.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [26]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [27]:
stack = 'MD589'
yaml_file = 'shape_params-aws.yaml'
scripts_dir = os.environ['REPO_DIR']

## Create Process Table
--script  shapeolopy_code/scripts/Cell_generator.py

In [21]:
@schema
class Process(dj.Computed):
    definition="""
    -> Structure
    -----
    size_positive_properties : int   #size of file
    size_negative_properties : int   #size of file
    size_positive_images     : int   #size of file
    size_negative_images     : int   #size of file
    """
    bucket = "mousebrainatlas-data"
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        for state in ['positive', 'negative']:
            item_name = state+'_s3_fp'
            img_name = state+'_img_fp'
            key_item = 'size_'+state+'_properties'
            key_img = 'size_'+state+'_images'
            s3_fp = (Structure & key).fetch1(item_name)
            img_fp = (Structure & key).fetch1(img_name)
            try:
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size/1000000)
                report2 = self.client.stat_object(self.bucket, img_fp)
                key[key_img] = int(report2.size/1000000)
            except:
                ! python3 $scripts_dir/Cell_generator.py $stack $struc $state $yaml_file
                upload_fp = 's3://'+self.bucket+'/'+s3_fp
                upload_img_fp = 's3://'+self.bucket+'/'+img_fp
                pkl_out_file = os.environ['ROOT_DIR']+s3_fp
                img_out_file = os.environ['ROOT_DIR']+img_fp
                ! aws s3 cp $pkl_out_file $upload_fp
                ! aws s3 cp $img_out_file $upload_img_fp
                print(pkl_out_file,upload_fp)
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size / 1000000)
                report2 = self.client.stat_object(self.bucket, img_fp)
                key[key_img] = int(report2.size/1000000)
        try:
            self.insert1(key)
        except:
            print('could not insert key=', key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [20]:
Process.drop()

`kui_diffusionmap`.`__process` (28 tuples)
Proceed? [yes, No]: yew
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [22]:
diffusion = Process()

In [23]:
diffusion.populate()#reserve_jobs=True

populating for  5N
populating for  6N
populating for  7N
populating for  7n
populating for  Amb
populating for  LC
populating for  LRt
populating for  Pn
populating for  Tz
populating for  VLL
populating for  RMC
populating for  SNC
populating for  SNR
populating for  3N
populating for  4N
populating for  Sp5I
populating for  Sp5O
populating for  Sp5C
populating for  PBG
populating for  10N
populating for  VCA
populating for  VCP
populating for  DC
populating for  AP
populating for  12N
populating for  RtTg
populating for  SC
2019-06-07 07:34:58,525	WARNING worker.py:1341 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-06-07 07:34:58,526	INFO node.py:497 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-07_07-34-58_525822_11505/logs.
2019-06-07 07:34:58,634	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:34772 t

/data/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_features/MD589/Properties/SC/MD589_SC_negative.pkl s3://mousebrainatlas-data/CSHL_cells_features/MD589/Properties/SC/MD589_SC_negative.pkl
populating for  IC
2019-06-07 08:13:28,219	WARNING worker.py:1341 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-06-07 08:13:28,219	INFO node.py:497 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-07_08-13-28_219269_24181/logs.
2019-06-07 08:13:28,326	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:35433 to respond...
2019-06-07 08:13:28,435	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:41354 to respond...
2019-06-07 08:13:28,436	INFO services.py:806 -- Starting Redis shard with 10.0 GB max memory.
2019-06-07 08:13:28,446	INFO node.py:511 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_20

In [28]:
Process.fetch(as_dict=True)

[OrderedDict([('id', 0),
              ('size_positive_properties', 51),
              ('size_negative_properties', 106),
              ('size_positive_images', 1752),
              ('size_negative_images', 3063)]),
 OrderedDict([('id', 1),
              ('size_positive_properties', 0),
              ('size_negative_properties', 6),
              ('size_positive_images', 16),
              ('size_negative_images', 136)]),
 OrderedDict([('id', 2),
              ('size_positive_properties', 98),
              ('size_negative_properties', 180),
              ('size_positive_images', 3366),
              ('size_negative_images', 4754)]),
 OrderedDict([('id', 3),
              ('size_positive_properties', 12),
              ('size_negative_properties', 186),
              ('size_positive_images', 207),
              ('size_negative_images', 4893)]),
 OrderedDict([('id', 4),
              ('size_positive_properties', 2),
              ('size_negative_properties', 21),
              ('size_po